# Exercise 1.1

This application works by reading a video file using cv2.VideoCapture() frame by frame. Each frame is then denoised by applying cv2.GaussianBlur(). Then, each frame is processed by an instance of a background subtractor initialized by cv2.createBackgroundSubtractorKNN() method. The resulting mask is thresholded by cv2.threshold() method, denoised by cv2.morphologyEx(), and passed to cv2.findContours() method to find the contours. The contours are then filtered by area. The resulting contours are then drawn on the original frame and displayed.

Install necessary packages:

In [1]:
%pip install opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


Import OpenCV library:

In [2]:
import cv2
import numpy as np

In [3]:
# The higher the value, the bigger a moving object should be to be detected
MIN_CONTOUR_AREA = 4000

In [4]:
# Open the sample video file:
video = cv2.VideoCapture("Traffic_Laramie_1.mp4")

# Ensure that the file was opened correctly:
assert video.isOpened(), "Can't open the video file"

# Initialize an instance of [Gaussian Mixture-based Background/Foreground Segmentation Algorithm](https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html):
bg_subtractor = cv2.createBackgroundSubtractorKNN(detectShadows=False, history=10000, dist2Threshold=900)

cv2.namedWindow("Cam")

# Read the video frame by frame:
while True:
    ret, frame = video.read()

    # Check for the last frame
    if not ret:
        break

    # blur the frame to remove noise
    blured_frame = cv2.GaussianBlur(frame, (5,5), 0)

    # Apply the background subtraction algorithm:
    fg_mask = bg_subtractor.apply(blured_frame)

    # Threshold the foreground mask to remove the shadows:
    _, fg_mask = cv2.threshold(fg_mask, 100, 255, cv2.THRESH_BINARY)

    # Apply morphological operations to remove corruptions:
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find the contours of the detected objects:
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out the contours that are too small to be a car:
    contours = [c for c in contours if cv2.contourArea(c) > MIN_CONTOUR_AREA]

    # Filter out the contours that in the upper half of the frame,
    # because we are only interested in the cars that are running along the Main Street:
    contours = [c for c in contours if cv2.boundingRect(c)[1] > frame.shape[0] / 2]

    # Convex hull to get clean contours without holes:
    contours = [cv2.convexHull(c) for c in contours]

    # Draw the contours for debugging purposes:
    # cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)

    # Prepare bounding boxes for the tracker
    bboxes = []

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)

        if y < frame.shape[0] / 2:
            continue

        bboxes.append([x, y, x+w, y+h])

    for [x,y, x2,y2] in bboxes:
        # Draw the bounding box around the detected object
        cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 1)

    # Display the resulting frame
    cv2.imshow("Cam", frame)

    # Press "q" to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyWindow("Cam")
        cv2.waitKey(1)
        break